<a href="https://colab.research.google.com/github/Saulo-Ferraz/PythonCodes/blob/main/Atv_CNN_Animais_Marinhos.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install tensorflow==2.15.0

In [ ]:
import tensorflow as tf
print(tf.__version__)

2.15.0


In [ ]:
import os
import gdown
import zipfile
import matplotlib.pyplot as plt
from keras.preprocessing.image import ImageDataGenerator
from keras.applications import NASNetLarge, EfficientNetV2L
from keras import layers, models, optimizers
from keras.utils import to_categorical
from sklearn.metrics import classification_report, confusion_matrix

In [ ]:
# Link de compartilhamento do Google Drive
zip_file_url = 'https://drive.google.com/uc?id=1syXLiu8GlZoyvSstb9tz_rBU3yBOp8Do'

# Caminho local para o arquivo zip
zip_file_path = '/content/dataset.zip'

# Função de Download
gdown.download(zip_file_url, zip_file_path, quiet=False)

Downloading...
From: https://drive.google.com/uc?id=1syXLiu8GlZoyvSstb9tz_rBU3yBOp8Do
To: /content/dataset.zip
100%|██████████| 19.5M/19.5M [00:00<00:00, 77.2MB/s]


'/content/dataset.zip'

In [ ]:
# Caminho para o diretório onde você deseja extrair as pastas
extracted_dir = '/content/dataset'

# Crie o diretório se não existir
os.makedirs(extracted_dir, exist_ok=True)

# Extrair o conteúdo do arquivo zip para o diretório de extração
with zipfile.ZipFile(zip_file_path, 'r') as zip_ref:
    zip_ref.extractall(extracted_dir)

# Lista os arquivos no diretório de extração
extracted_files = os.listdir(extracted_dir)
print("Arquivos extraídos:", extracted_files)

# Diretórios de treinamento e validação
train_dir = os.path.join(extracted_dir)
validation_dir = os.path.join(extracted_dir)

Arquivos extraídos: ['fishes', 'turtles', 'jellyfish', 'sharks']


In [ ]:
# Configurações
batch_size = 200
img_size = (224, 224)  # Tamanho de imagem adequado para os modelos de transfer learning
num_classes = len(os.listdir(train_dir)) # Número de classes

# Data augmentation para os dados de treinamento
train_datagen = ImageDataGenerator(
    rescale=1.0/255.0,
    rotation_range=40,
    width_shift_range=0.2,
    height_shift_range=0.2,
    shear_range=0.2,
    zoom_range=0.2,
    horizontal_flip=True,
    fill_mode='nearest'
)

# Data augmentation para os dados de validação
validation_datagen = ImageDataGenerator(rescale=1.0/255.0)

# Criação dos geradores de dados de treinamento e validação
train_generator = train_datagen.flow_from_directory(
    train_dir,
    target_size=img_size,
    batch_size=batch_size,
    class_mode='categorical',
    shuffle=True
)

validation_generator = validation_datagen.flow_from_directory(
    validation_dir,
    target_size=img_size,
    batch_size=batch_size,
    class_mode='categorical',
    shuffle=False
)

Found 2189 images belonging to 4 classes.
Found 2189 images belonging to 4 classes.


In [ ]:
# Modelo 1: CNN personalizada
model1 = models.Sequential()
model1.add(layers.Conv2D(16, (3, 3), activation='relu', input_shape=(img_size[0], img_size[1], 3)))
model1.add(layers.MaxPooling2D((2, 2)))
model1.add(layers.Conv2D(32, (3, 3), activation='relu'))
model1.add(layers.MaxPooling2D((2, 2)))
model1.add(layers.Flatten())
model1.add(layers.Dense(128, activation = 'relu'))
model1.add(layers.Dense(num_classes, activation='softmax'))

model1.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])

In [ ]:
model1.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 conv2d (Conv2D)             (None, 222, 222, 16)      448       
                                                                 
 max_pooling2d (MaxPooling2  (None, 111, 111, 16)      0         
 D)                                                              
                                                                 
 conv2d_1 (Conv2D)           (None, 109, 109, 32)      4640      
                                                                 
 max_pooling2d_1 (MaxPoolin  (None, 54, 54, 32)        0         
 g2D)                                                            
                                                                 
 flatten (Flatten)           (None, 93312)             0         
                                                                 
 dense (Dense)               (None, 128)               1

In [ ]:
# Modelo 2: CNN personalizada com hiperparâmetros diferentes
model2 = models.Sequential()
model2.add(layers.Conv2D(32, (3, 3), activation='relu', input_shape=(img_size[0], img_size[1], 3)))
model2.add(layers.MaxPooling2D((2, 2)))
model2.add(layers.Conv2D(64, (3, 3), activation='relu'))
model2.add(layers.MaxPooling2D((2, 2)))
model2.add(layers.Flatten())
model2.add(layers.Dense(128, activation = 'relu'))
model2.add(layers.Dense(num_classes, activation='softmax'))

model2.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])

In [ ]:
model2.summary()

Model: "sequential_1"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 conv2d_2 (Conv2D)           (None, 222, 222, 32)      896       
                                                                 
 max_pooling2d_2 (MaxPoolin  (None, 111, 111, 32)      0         
 g2D)                                                            
                                                                 
 conv2d_3 (Conv2D)           (None, 109, 109, 64)      18496     
                                                                 
 max_pooling2d_3 (MaxPoolin  (None, 54, 54, 64)        0         
 g2D)                                                            
                                                                 
 flatten_1 (Flatten)         (None, 186624)            0         
                                                                 
 dense_2 (Dense)             (None, 128)              

In [ ]:
base_model1 = NASNetLarge(weights='imagenet', include_top=False, input_shape=(img_size[0], img_size[1], 3))
model_tl1 = models.Sequential()
model_tl1.add(base_model1)
model_tl1.add(layers.Conv2D(32, (3, 3), activation='relu'))
model_tl1.add(layers.Flatten())
model_tl1.add(layers.Dense(128, activation='relu'))
model_tl1.add(layers.Dense(num_classes, activation='softmax'))

for layer in base_model1.layers:
    layer.trainable = False

model_tl1.compile(loss='categorical_crossentropy', optimizer=optimizers.Adam(learning_rate=1e-4), metrics=['accuracy'])

In [ ]:
model_tl1.summary()

Model: "sequential_2"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 NASNet (Functional)         (None, 7, 7, 4032)        84916818  
                                                                 
 conv2d_4 (Conv2D)           (None, 5, 5, 32)          1161248   
                                                                 
 flatten_2 (Flatten)         (None, 800)               0         
                                                                 
 dense_4 (Dense)             (None, 128)               102528    
                                                                 
 dense_5 (Dense)             (None, 4)                 516       
                                                                 
Total params: 86181110 (328.75 MB)
Trainable params: 1264292 (4.82 MB)
Non-trainable params: 84916818 (323.93 MB)
_________________________________________________________________


In [ ]:
base_model2 = EfficientNetV2L(weights='imagenet', include_top=False, input_shape=(img_size[0], img_size[1], 3))
model_tl2 = models.Sequential()
model_tl2.add(base_model2)
model_tl2.add(layers.Conv2D(32, (3, 3), activation='relu'))
model_tl2.add(layers.Flatten())
model_tl2.add(layers.Dense(128, activation='relu'))
model_tl2.add(layers.Dense(num_classes, activation='softmax'))

for layer in base_model2.layers:
    layer.trainable = False

model_tl2.compile(loss='categorical_crossentropy', optimizer=optimizers.Adam(learning_rate=1e-4), metrics=['accuracy'])

473176280/473176280 [==============================] - 6s 0us/step


In [ ]:
model_tl2.summary()

Model: "sequential_3"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 efficientnetv2-l (Function  (None, 7, 7, 1280)        117746848 
 al)                                                             
                                                                 
 conv2d_5 (Conv2D)           (None, 5, 5, 32)          368672    
                                                                 
 flatten_3 (Flatten)         (None, 800)               0         
                                                                 
 dense_6 (Dense)             (None, 128)               102528    
                                                                 
 dense_7 (Dense)             (None, 4)                 516       
                                                                 
Total params: 118218564 (450.97 MB)
Trainable params: 471716 (1.80 MB)
Non-trainable params: 117746848 (449.17 MB)
_____

In [ ]:
model1.fit(train_generator, epochs=1, validation_data=validation_generator)

11/11 [==============================] - 132s 12s/step - loss: 9.4061 - accuracy: 0.2759 - val_loss: 1.9081 - val_accuracy: 0.2773


In [ ]:
model2.fit(train_generator, epochs=1, validation_data=validation_generator)

11/11 [==============================] - 244s 22s/step - loss: 10.6996 - accuracy: 0.2768 - val_loss: 1.3223 - val_accuracy: 0.3586


In [ ]:
model_tl1.fit(train_generator, epochs=1, validation_data=validation_generator)

11/11 [==============================] - 2022s 191s/step - loss: 0.3563 - accuracy: 0.8780 - val_loss: 0.2231 - val_accuracy: 0.9269


In [ ]:
model_tl2.fit(train_generator, epochs=1, validation_data=validation_generator)

11/11 [==============================] - 2295s 215s/step - loss: 1.4282 - accuracy: 0.2430 - val_loss: 1.3803 - val_accuracy: 0.3280


> > #  Análise dos Resultados



---



---





1.   O Modelo NASNetLarge obteve a maior acurácia porém foi o que levou mais tempo para terminar.
2.   O 2° Modelo de CNN obteve um resultado pouco maior que o 1° mesmo com uma maior quantidade de hiperparamêtros.
3.  O tempo que os modelos de TL ( Transfer Learning ) levaram foi substâncialmente maior que os de CNN.
4.  A quantidade de paramêtros não treináveis nos modelos de TL é absurdamente grande.
5.  Se fez necessária a instalação de uma versão mais atualizada do Tensorflow pois na 2.14.0 os modelos não obtiveram sucesso no treino.



Dupla: Saulo Raphael e Athos Pugliese